## Overall Prediction Observation


In [7]:
# Define variables
IS_FILTERED = True
ARCHIVED = None

SHOW_HEADER = True
SHOW_LAST_APPEARANCE = None

In [8]:
# Import libraries
import sys
sys.path.append('../../../prediction')

import numpy as np
import matplotlib.pyplot as plt

from putils.observation import compute_rmse, compute_mae, compute_mape, load_data_from_tuned_folder

In [9]:
# Load data
dfs = {
    "l1": load_data_from_tuned_folder("l1", last_appearance=SHOW_LAST_APPEARANCE),
    "l2": load_data_from_tuned_folder("l2", last_appearance=SHOW_LAST_APPEARANCE),
    "l3": load_data_from_tuned_folder("l3", last_appearance=SHOW_LAST_APPEARANCE),
}

# Make L1 and L2 data have the same length with L3 if SHOW_LAST_APPEARANCE = None
if SHOW_LAST_APPEARANCE is None:
    dfs["l1"] = dfs["l1"].tail(len(dfs["l3"]))
    dfs["l2"] = dfs["l2"].tail(len(dfs["l3"]))

# Indicate which columns to ignore
IGNORED_COLS = ["Time", "Actual"] # No need to include "FormattedTime" because it is used as index
if IS_FILTERED: IGNORED_COLS.append("Raw")

In [10]:
# Compute RMSE, MAE, and MAPE for each model
for key, df in dfs.items():
    print("Layer", key)
    for column in df.columns:
        if column not in IGNORED_COLS:
            actual = df["Raw"] if IS_FILTERED else df["Actual"]
            rmse = compute_rmse(actual, df[column])
            mape = compute_mape(actual, df[column])
            mae = compute_mae(actual, df[column])
            print(f"{column} - RMSE: {rmse:.6f}, MAE: {mae:.4f}, MAPE: {mape:.2f}%")
    print("------------------")

Layer l1
ARIMA - RMSE: 3657.492956, MAE: 2828.5929, MAPE: 59.27%
SARIMA - RMSE: 3570142765.434386, MAE: 206705187.0240, MAPE: 6620079.09%
RNN - RMSE: 2787.250679, MAE: 2179.4873, MAPE: 48.13%
LSTM - RMSE: 2826.799096, MAE: 2221.6232, MAPE: 49.28%
CNN - RMSE: 2897.421691, MAE: 2289.3315, MAPE: 50.15%
GRU - RMSE: 2795.238104, MAE: 2187.4274, MAPE: 48.44%
TCN - RMSE: 3069.801164, MAE: 2440.4078, MAPE: 53.72%
------------------
Layer l2
LINEAR_REGRESSION - RMSE: 2775.283776, MAE: 2169.9336, MAPE: 48.06%
RANDOM_FOREST - RMSE: 2800.649139, MAE: 2192.6197, MAPE: 48.35%
FEEDFORWARD_NEURAL_NETWORK - RMSE: 2779.803686, MAE: 2173.5007, MAPE: 47.93%
------------------
Layer l3
Predicted - RMSE: 2777.587717, MAE: 2173.0116, MAPE: 48.02%
------------------


In [11]:
l2_predictions = dfs["l2"].drop(columns=IGNORED_COLS)
average_predictions = l2_predictions.mean(axis=1)
rmse = compute_rmse(dfs["l2"]["Raw"], average_predictions)
print(f"Average RMSE: {rmse:.6f}")

Average RMSE: 2778.211388
